In [ ]:
# https://www.glowscript.org/docs/VPythonDocs/canvas.html
# https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/
# https://vpython.org/
!pip install vpython

In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
df = pd.read_csv("mb.csv")
df.set_index(["ix", "iy", "iz"], drop=False, inplace=True)
palette = sns.color_palette("pastel", 20)
palette[0]

FileNotFoundError: [Errno 2] No such file or directory: 'mb.csv'

In [ ]:
df = pd.read_csv("mb.csv")
df

In [ ]:
# set index
df.set_index(["ix", "iy", "iz"], drop=False, inplace=True)

In [ ]:
# 0 - 8
# 1.2 - 5.6 --> (0 - 4.4) * (8/4.4) --> 0 - 8  --> int((x - 1.2) * (8 / 4.4))

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.color_palette.html
"""Paired, magma, crest, viridis, Set2, husl, hls, pastel, muted"""
palette = sns.color_palette("pastel", 20)
palette[0]

In [ ]:
# Visualizador 3D
import math
import pandas as pd
import numpy as np
import seaborn as sns
from vpython import *

df = pd.read_csv("mb.csv")
df.set_index(["ix", "iy", "iz"], drop=False, inplace=True)
palette = sns.color_palette("pastel", 20)

def visor3D(df, campo, range_color, df2):
    list_blocks = []
    scene.width = 800
    scene.height = 350

    df = df[["ix", "iy", "iz", campo]]
    minx, maxx = df.ix.min(), df.ix.max()
    miny, maxy = df.iy.min(), df.iy.max()
    minz, maxz = df.iz.min(), df.iz.max()
    mincampo, maxcampo = df[campo].min(), df[campo].max()
    
    scene.title = ""
    scene.center = vec((minx + maxx)/2, (miny + maxy)/2, (minz + maxz)/2)
    scene.camera.up = vec(1, 0, 0)
    difcampo = maxcampo - mincampo

    df["color"] = df[campo].apply(lambda x: int((x - mincampo) * (range_color / difcampo)))
    
    for pos in df.index:
        list_blocks.append(box(pos=vec(*pos), color=vec(*palette[df.loc[pos, "color"]]))) # df.color.loc[pos]

    df.reset_index(drop=True, inplace=True)
    #print(df)
    
    def mostrar_campo_filter(x):
        wtcampo.text = f"{x.value}"
        mostrar = df[df[campo] > x.value].index
        ocultar = df[~(df[campo] > x.value)].index
        for i in mostrar:
            list_blocks[i].visible = True
        for i in ocultar:
            list_blocks[i].visible = False
            
    def mostrar_x_filter(x):
        wtx.text = f"{x.value}"
        if check_mix.checked:
            mostrar = df[df[campo] > slider_campo.value]
            mostrar = mostrar[mostrar["ix"] == int(x.value)].index
        else:
            mostrar = df[df["ix"] == int(x.value)].index

        ocultar = df[~(df["ix"] == int(x.value))].index
        for i in mostrar:
            list_blocks[i].visible = True
        for i in ocultar:
            list_blocks[i].visible = False
            
    def mostrar_y_filter(x):
        wty.text = f"{x.value}"
        if check_mix.checked:
            mostrar = df[df[campo] > slider_campo.value]
            mostrar = mostrar[mostrar["iy"] == int(x.value)].index
        else:
            mostrar = df[df["iy"] == int(x.value)].index
        ocultar = df[~(df["iy"] == int(x.value))].index
        for i in mostrar:
            list_blocks[i].visible = True
        for i in ocultar:
            list_blocks[i].visible = False
            
    def mostrar_z_filter(x):
        wtz.text = f"{x.value}"
        if check_mix.checked:
            mostrar = df[df[campo] > slider_campo.value]
            mostrar = mostrar[mostrar["iz"] == int(x.value)].index
        else:
            mostrar = df[df["iz"] == int(x.value)].index
        ocultar = df[~(df["iz"] == int(x.value))].index
        for i in mostrar:
            list_blocks[i].visible = True
        for i in ocultar:
            list_blocks[i].visible = False
    
    def pass_check(x):
        pass
    
    def click_block():
        block = scene.mouse.pick
        if block:
            cor = block.pos.value
            most = df2.loc[(int(cor[0]), int(cor[1]), int(cor[2])), ["kcu", "period_mined"]]
            scene.title = f"{most}"
    
    scene.bind("mousedown", click_block)
    
    scene.caption = f"Filtro por {campo} {mincampo*0.9}"
    slider_campo = slider(min=mincampo*0.9, max=maxcampo * 1.1, value=-1, bind=mostrar_campo_filter, lenght=220)
    wtcampo = wtext(text=f"{maxcampo * 1.1}")
    
    scene.append_to_caption("\n")
    check_mix = checkbox(text="filtro mixto", bind=pass_check)
    scene.append_to_caption("\n")
    
    scene.append_to_caption("\nCorte X")
    slider_x = slider(min=minx - 1, max=maxx + 1, value=minx - 1, bind=mostrar_x_filter, lenght=220)
    wtx = wtext(text=f"{minx - 1}")
    
    scene.append_to_caption("\n")
    
    scene.append_to_caption("\nCorte Y")
    slider_y = slider(min=miny - 1, max=maxy + 1, value=miny - 1, bind=mostrar_y_filter, lenght=220)
    wty = wtext(text=f"{miny - 1}")
    
    scene.append_to_caption("\n")
    
    scene.append_to_caption("\nCorte Z")
    slider_z = slider(min=minz - 1, max=maxz + 1, value=minz - 1, bind=mostrar_z_filter, lenght=220)
    wtz = wtext(text=f"{minz - 1}")

df2 = df.copy(True)
visor3D(df[df.kcu > 0], "kcu", 8, df2)